In [2]:
# some imports
import pygsti
import pygsti.modelmembers as mm
import numpy as np

/Users/pablovinasmartinez/opt/anaconda3/envs/coloured_GST/lib/python3.12/site-packages/pygsti/baseobjs/opcalc/__init__.py:31: UserWarning: 
An optimized Cython-based implementation of `pygsti.baseobjs.opcalc` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

  _warnings.warn(warn_msg)
/Users/pablovinasmartinez/opt/anaconda3/envs/coloured_GST/lib/python3.12/site-packages/pygsti/circuits/circuitparser/__init__.py:60: UserWarning: 
An optimized Cython-based implementation of `pygsti.circuits.circuitparser` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNI

Our model is constructed from two single-qubit operations. These operations are integrated and combined to create a two-qubit operation, with each single-qubit gate acting on its respective qubit.

In [3]:
mdl_complete = pygsti.models.ExplicitOpModel(['Q0', 'Q1'],'pp')

# Single qubit operations

G1 = np.array([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 0,-1],
               [0, 0, 1, 0]], 'd')

G2 = np.array([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0,-1, 0],
               [0, 0, 0,-1]], 'd')

Op1, Op2 = mm.operations.FullArbitraryOp(G1), mm.operations.FullArbitraryOp(G2)

# Composition
mdl_complete.operations[('G1G2')] = mm.operations.ComposedOp(
                                        (mm.operations.EmbeddedOp(['Q0', 'Q1'], ['Q0'], Op1),
                                        mm.operations.EmbeddedOp(['Q0', 'Q1'], ['Q1'], Op2))
                                        )
# Check that the operation is correct
print(mdl_complete[('G1G2')])

Composed operation of 2 factors:
Factor 0:
Embedded operation with full dimension 16 and state space QubitSpace(('Q0', 'Q1'))
 that embeds the following 4-dimensional operation into acting on the ('Q0',) space
FullArbitraryOp with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0   0-1.00
   0   0 1.00   0
Factor 1:
Embedded operation with full dimension 16 and state space QubitSpace(('Q0', 'Q1'))
 that embeds the following 4-dimensional operation into acting on the ('Q1',) space
FullArbitraryOp with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0-1.00   0
   0   0   0-1.00



We attempt to identify a set of germs using the `find_germs()` function. This is purely for experimental reasons, given that the gate set is limited to a single operation and is not even IC. The process fails due to the presence of a composite operation in the model, which is not supported.

In [4]:
import pygsti.algorithms.germselection as germsel

germs = germsel.find_germs(mdl_complete)

TypeError: unsupported operand type(s) for *: 'float' and 'ComposedOp'

To address this issue, we can modify the `randomize_with_unitary()` function. By altering the final line to `gate -> gate.to_dense()`, we ensure that the `.dot` product operation is compatible with the matrix format.

In [5]:
import numpy as _np
import scipy as _scipy
from pygsti.modelmembers import operations as _op
from pygsti.tools import optools as _ot


def randomize_with_unitary(self, scale, seed=None, rand_state=None):
        
        """
        Create a new model with random unitary perturbations.

        Apply a random unitary to each element of a model, and return the
        result, without modifying the original (this) model. This method
        works on Model as long as the dimension is a perfect square.

        Parameters
        ----------
        scale : float
            maximum element magnitude in the generator of each random unitary
            transform.

        seed : int, optional
            if not None, seed numpy's random number generator with this value
            before generating random depolarizations.

        rand_state : numpy.random.RandomState
            A RandomState object to generate samples from. Can be useful to set
            instead of `seed` if you want reproducible distribution samples
            across multiple random function calls but you don't want to bother
            with manually incrementing seeds between those calls.

        Returns
        -------
        Model
            the randomized Model
        """

        if rand_state is None:
            rndm = _np.random.RandomState(seed)
        else:
            rndm = rand_state

        op_dim = self.state_space.dim
        unitary_dim = int(round(_np.sqrt(op_dim)))
        assert(unitary_dim**2 == op_dim), \
            "Model dimension must be a perfect square, %d is not" % op_dim

        mdl_randomized = self.copy()

        for opLabel, gate in self.operations.items():
            randMat = scale * (rndm.randn(unitary_dim, unitary_dim)
                               + 1j * rndm.randn(unitary_dim, unitary_dim))
            randMat = _np.transpose(_np.conjugate(randMat)) + randMat
            # make randMat Hermetian: (A_dag + A)^dag = (A_dag + A)
            randUnitary = _scipy.linalg.expm(-1j * randMat)

            randOp = _ot.unitary_to_superop(randUnitary, self.basis)

            mdl_randomized.operations[opLabel] = _op.FullArbitraryOp(_np.dot(randOp, gate.to_dense())) # THIS IS THE UPDATED LINE!!

        #Note: this function does NOT randomize instruments

        return mdl_randomized


pygsti.models.ExplicitOpModel.randomize_with_unitary = randomize_with_unitary

Now `find_germs()` works

In [6]:
germs = germsel.find_germs(mdl_complete)

Initial Length Available Germ List: 1
Length Available Germ List After Deduping: 1
Length Available Germ List After Dropping Random Fraction: 1
Length Available Germ List After Adding Back In Forced Germs: 1
Memory estimate of 0.0 GB for all-Jac mode.
Memory estimate of 0.0 GB for single-Jac mode.
Using greedy algorithm.
Constructed germ set:
['G1G2']
Score: major=-28.0 minor=28.000000000000004, N: 28
